In [1]:
%matplotlib widget

import logging
import time

import cftime
import httpx
import matplotlib.pyplot as plt
import netCDF4
import pandas as pd
import shapely.io
import sqlmodel
from sqlalchemy import (
    bindparam, 
    func
)

from arpav_ppcv import (
    database as db,
    operations,
)
from arpav_ppcv.config import get_settings
from arpav_ppcv.schemas.base import (
    CoverageDataSmoothingStrategy,
    ObservationDataSmoothingStrategy,
    ObservationAggregationType,
    Season,
)
from arpav_ppcv.schemas import coverages
from arpav_ppcv.schemas.coverages import CoverageInternal

logging.basicConfig(level=logging.DEBUG)
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("httpcore").setLevel(logging.WARNING)
logging.getLogger("matplotlib").setLevel(logging.WARNING)

settings = get_settings()
session = sqlmodel.Session(db.get_engine(settings))
http_client = httpx.Client()

In [2]:
coverage_identifier = "tas_seasonal_absolute_model_ensemble-annual-model_ensemble-tas-absolute-rcp26-DJF"
point_coords = "POINT(11.5469 44.9524)"
date_range = "../.."

cov = CoverageInternal(
    configuration=db.get_coverage_configuration_by_coverage_identifier(session, coverage_identifier), 
    identifier=coverage_identifier
)

In [3]:
operations.get_related_uncertainty_coverage_configurations(session, cov)[1].identifier

'tas_seasonal_absolute_model_ensemble_upper_uncertainty-annual-model_ensemble-tas-absolute-rcp26-upper_bound-DJF'

In [6]:
cov.configuration.uncertainty_lower_bounds_coverage_configuration

CoverageConfiguration(description_english='Average daily air temperature near the ground', color_scale_max=32.0, description_italian="Temperatura media giornaliera dell'aria vicino al suolo", observation_variable_id=None, netcdf_main_dataset_name='tas_stddown', observation_variable_aggregation_type=None, thredds_url_pattern='ensymbc/std/clipped/tas_stddown_{scenario}_{year_period}_ts19762100_ls_VFVG.nc', uncertainty_lower_bounds_coverage_configuration_id=None, name='tas_seasonal_absolute_model_ensemble_lower_uncertainty', wms_main_layer_name='tas_stddown', uncertainty_upper_bounds_coverage_configuration_id=None, id=UUID('37ec5342-7eba-4d99-831e-b8e027577ec0'), unit='ºC', display_name_english='Mean temperature', palette='default/seq-YlOrRd', display_name_italian='Temperatura media', color_scale_min=-3.0, coverage_id_pattern='{name}-{aggregation_period}-{climatological_model}-{climatological_variable}-{measure}-{scenario}-{uncertainty_type}-{year_period}')

In [4]:
[c.identifier for c in operations.get_related_coverages(cov)]

['tas_seasonal_absolute_model_ec_earth_cclm4_8_17-annual-ec_earth_cclm_4_8_17-tas-absolute-rcp26-DJF',
 'tas_seasonal_absolute_model_ec_earth_racmo22e-annual-ec_earth_racmo22e-tas-absolute-rcp26-DJF',
 'tas_seasonal_absolute_model_ec_earth_rca4-annual-ec_earth_rca4-tas-absolute-rcp26-DJF',
 'tas_seasonal_absolute_model_hadgem2_es_racmo22e-annual-hadgem2_racmo22e-tas-absolute-rcp26-DJF',
 'tas_seasonal_absolute_model_mpi_esm_lr_remo2009-annual-mpi_esm_lr_remo2009-tas-absolute-rcp26-DJF']